In [ ]:
import requests  

url = 'http://books.toscrape.com/catalogue/page-1.html'
response = requests.get(url)
response

In [ ]:
from bs4 import BeautifulSoup

# распарсили страничку в дерево 
tree = BeautifulSoup(response.content, 'html.parser')  # содержимое запроса в байтах - .content
tree
"""
Использование .content обеспечивает доступ к чистым байтам ответного пейлоада, 
то есть к любым данным в теле запроса. Однако, зачастую требуется конвертировать
полученную информацию в строку в кодировке UTF-8. response делает это при помощи .text.
"""

In [ ]:
response.encoding # узнаю кодировку

In [ ]:
books = tree.find_all('article', {'class' : 'product_pod'})
books[0]     # вся инфа о первой книге

In [ ]:
t = books[0].h3.a.get('title')
h = books[0].h3.a.get('href')
print (t)
print ('http://books.toscrape.com/catalogue/' + h)

In [ ]:
books[0].p.get('class')[1]

In [ ]:
books[0].find('p', {'class': 'price_color'}).text

In [ ]:
books[0].find('p', {'class': 'instock availability'}).text.split()

In [ ]:
info = []
for i in range(20):
    d = {'title': books[i].h3.a.get('title'),
        'url': 'https://books.toscrape.com/catalogue/' + books[i].h3.a.get('href'),
        'rating': books[i].p.get('class')[1],
        'price': books[i].find('p', {'class': 'price_color'}).text,
        'instock': books[i].find('p', {'class': 'instock availability'}).text.split()}
    info.append(d)
info

In [ ]:
num_page = 17916#2913591
f'http://books.toscrape.com/catalogue/page-{num_page}.html'  # для перебора по страницам

In [ ]:
def get_page(num_page):  # функция перебора книг на странице
    url = f'http://books.toscrape.com/catalogue/page-{num_page}.html'
    response = requests.get(url)
    
    tree = BeautifulSoup(response.content, 'html.parser')
    books = tree.find_all('article', {'class' : 'product_pod'})
    info = []
    for i in range(20):       # на каждой странице 20 книг
        d = {'title': books[i].h3.a.get('title'),
        'url': 'https://books.toscrape.com/catalogue/' + books[i].h3.a.get('href'),
        'rating': books[i].p.get('class')[1],
        'price': books[i].find('p', {'class': 'price_color'}).text,
        'instock': books[i].find('p', {'class': 'instock availability'}).text.split()}
        info.append(d)
    return info

In [ ]:
books_list = []
for i in range(1, 51):  # всего 50 страниц
    books_info = get_page(i)  # собираю инфу по каждой книге через функцию get_page
    books_list.extend(books_info)


In [ ]:
import pandas as pd
df = pd.DataFrame(books_list)
df

In [ ]:
df.columns = ['Book', 'Web', 'Rating', 'price', 'Avail-ty']
df

In [ ]:
def get_price(price):               # данные по ценам конвертированы в float
    return float(price[1:])   # срез, берем значение со второго символа, т.к. первый это £
df['price'] = df['price'].apply(get_price)
df

In [ ]:
df['price'].hist(); # bins=20

In [ ]:
import os
os.getcwd()   # смотрю куда сейчас сохраняются файлы

In [ ]:
os.chdir('C:\\Users\......')    # прописала адрес сохранения
os.getcwd()

In [ ]:
df.to_excel('Books.xlsx', index=False)        # сохранила таблицу в эксель на рабочий стол